# Compare total number of reads quantified/counted for different methods

* htseq: we mapped reads against the genome and then counted genes using htseq. htseq counts genes that overlap regions in the genome defined by a GTF file. As will be shown later in this notebook, while the genomes had high mapping rates (not shown), the number of reads that counted toward a feature annotated in the GTF file was pretty low. This led us to explore other options, namely using the transcriptome to count reads.
* salmon & a tx2gene file:  We observed higher mapping rates against the transcriptome, so we worked to create a `tx2gene` file, as differential expression analysis is generally more accurate at the gene level than the isoform level. We created this by using a splice-aware long read aligner (ultra) to map the assembled transcripts to the genome guided by the GTF file (uses minimap in the absence of GTF coordinates to splice on). We then assigned transcripts gene annotations by determining which genes they overlapped with using the mapping & GTF coordinates. While 91% of transcripts mapped, about half (approx 500k) transcripts were annotated as genes in this way. 
* salmon & a tx2gene file + transcripts with no gene annotations: We also augmented this tx2gene file with transcripts alone that didn't receive gene annotations. Presumably some fraction of reads map to these transcripts, so we back-added them to the tx2gene file to count these transcripts too.
* salmon with transcripts only: This approach to counting should be redundant with the one above, but we included it just to check (spoiler, it's redundant).

## notbook setup

In [1]:
library(tidyverse)
library(tximport)
library(jsonlite)

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘jsonlite’


The following object is masked from ‘package:purrr’:

    flatten




In [2]:
setwd("..")

## functions

In [3]:
read_fastp_json <- function(file_path) {
  # read the json file
  json_data <- fromJSON(file_path)
  
  # extract num reads after filtering
  total_reads_after_filtering <- json_data$summary$after_filtering$total_reads
  
  # return a data frame
  return(data.frame(sample_name = gsub(".json", "", basename(file_path)),
                    total_reads = total_reads_after_filtering))
}

## read in data

In [4]:
# parse fastp log files to get accurate total read counts after qc 
fastp <- Sys.glob("outputs/read_qc/fastp/*json") %>%
  map_dfr(read_fastp_json)

In [5]:
# read in htseq data frame and remove summary rows
htseq <- read_tsv("outputs/counts/raw_counts.tsv", show_col_types = F) %>%
  filter(!grepl("^__", gene)) # filter out summary rows produced by htseq

In [6]:
# create a vector of column names to use for all salmon counts
salmon_colnames <- gsub("_quant", "", basename(gsub("/quant.sf", "", Sys.glob("outputs/quantification/*/quant.sf"))))

In [8]:
# read in salmon counts, output just the transcript counts without gene-level summarization
salmon_tx <- tximport(files = Sys.glob("outputs/quantification/*/quant.sf"),
                      type = "salmon", txOut = TRUE)
salmon_tx <- salmon_tx$counts
colnames(salmon_tx) <- salmon_colnames

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
25 
26 
27 
28 
29 
30 
31 
32 
33 
34 
35 
36 
37 
38 
39 
40 
41 
42 
43 
44 
45 
46 
47 
48 
49 
50 
51 
52 
53 
54 
55 




In [10]:
# read in the tx2gene file created by assigning transcripts gene names they overlapped with when mapped to to the reference genome
# restore missing transcripts that don't have gene assignments and that didn't map to the genome
tx2gene <- read_tsv("~/github/2023-amblyomma-americanum-txome-assembly/sandbox/try_ultra/Amblyomma_americanum_filtered_assembly.tx2gene_by_gtf.tsv",
                    skip = 1, col_names = c("tx", "gene"), show_col_types = FALSE)
missing_tx <- rownames(salmon_tx)[!rownames(salmon_tx) %in% tx2gene$tx]
missing_tx <- data.frame(tx = missing_tx, gene = missing_tx)
tx2gene <- bind_rows(tx2gene, missing_tx)

In [11]:
# read in the salmon counts using the full tx2gene file with both genes and transcripts represented
salmon_gx_tx <- tximport(files = Sys.glob("outputs/quantification/*/quant.sf"),
                         type = "salmon", txOut = FALSE, tx2gene = tx2gene)
salmon_gx_tx <- salmon_gx_tx$counts
colnames(salmon_gx_tx) <- salmon_colnames

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
25 
26 
27 
28 
29 
30 
31 
32 
33 
34 
35 
36 
37 
38 
39 
40 
41 
42 
43 
44 
45 
46 
47 
48 
49 
50 
51 
52 
53 
54 
55 


summarizing abundance

summarizing counts

summarizing length



In [12]:
# filter the tx2gene data.frame to only keep transcripts assigned to genes
tx2gene_gx_only <- tx2gene %>%
  filter(grepl(pattern = "evm", x = gene))

In [13]:
# read in salmon results and only keep counts that mapped to transcripts with gene assignments
salmon_gx_only <- tximport(files = Sys.glob("outputs/quantification/*/quant.sf"),
                           type = "salmon", txOut = FALSE, 
                           tx2gene = tx2gene_gx_only)
salmon_gx_only <- salmon_gx_only$counts
colnames(salmon_gx_only) <- salmon_colnames

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
25 
26 
27 
28 
29 
30 
31 
32 
33 
34 
35 
36 
37 
38 
39 
40 
41 
42 
43 
44 
45 
46 
47 
48 
49 
50 
51 
52 
53 
54 
55 


transcripts missing from tx2gene: 617596

summarizing abundance

summarizing counts

summarizing length



## calculate the total number of mapped reads for each method

In [14]:
sum_salmon_gx_only <- colSums(salmon_gx_only)
sum_salmon_tx <- colSums(salmon_tx)
sum_salmon_gx_tx <- colSums(salmon_gx_tx)
sum_htseq <- colSums(htseq[ , -1])

In [15]:
# convert results to data.frames
mapping_sums_salmon <- data.frame(sample_name = names(sum_salmon_gx_only),
                                  salmon_gx_only = sum_salmon_gx_only, 
                                  salmon_tx = sum_salmon_tx, 
                                  salmon_gx_tx = sum_salmon_gx_tx)
mapping_sums_htseq <- data.frame(sample_name = names(sum_htseq),
                                 htseq = sum_htseq)

## convert raw counts to percentages

In [18]:
combined <- left_join(fastp, mapping_sums_salmon, by = "sample_name") %>%
  left_join(mapping_sums_htseq, by = "sample_name") %>%
  mutate(p_salmon_gx_only = salmon_gx_only / total_reads,
         p_salmon_gx_tx = salmon_gx_tx / total_reads,
         p_salmon_tx = salmon_tx /total_reads,
         p_htseq = htseq / total_reads)

In [24]:
combined

sample_name,total_reads,salmon_gx_only,salmon_tx,salmon_gx_tx,htseq,p_salmon_gx_only,p_salmon_gx_tx,p_salmon_tx,p_htseq
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAFF,191306,22065.41,35707.01,35707.01,8101,0.11534090,0.18664868,0.18664868,0.04234577
AAFM,101672,16023.11,20560.01,20560.01,6400,0.15759605,0.20221900,0.20221900,0.06294752
AAUF,704018,23508.16,45493.01,45493.01,7301,0.03339142,0.06461909,0.06461909,0.01037047
AAUM,156290,38687.30,49328.97,49328.97,11161,0.24753534,0.31562460,0.31562460,0.07141212
AmbamSG7-11d,88372472,24349215.75,40773097.91,40773097.91,7131686,0.27552942,0.46137781,0.46137781,0.08070031
AmbamSG72-144h,99163790,26055582.29,45126079.96,45126079.96,7914394,0.26275299,0.45506611,0.45506611,0.07981133
AmbamSGunfed,104891872,22282481.44,45681707.96,45681707.96,6030717,0.21243287,0.43551237,0.43551237,0.05749461
AmbameSG12-18h,50175186,14832841.80,22922332.14,22922332.14,5236959,0.29562106,0.45684598,0.45684598,0.10437348
Ecoli_12h_1,43216570,16769004.38,20688579.83,20688579.83,4725147,0.38802257,0.47871869,0.47871869,0.10933647


## calculate mean mapping percentage for each method

In [25]:
round(colMeans(combined[ , -1], na.rm = T), digits = 2)

# the columns that start with `p_*` are percents (actually fractions) for each counting strategy

total_reads   salmon_gx_only        salmon_tx     salmon_gx_tx 
     33492252.27      13196875.76      17093211.08      17093211.08 
           htseq p_salmon_gx_only   p_salmon_gx_tx      p_salmon_tx 
      3844217.02             0.35             0.46             0.46 
         p_htseq 
            0.10

## summary

As can be seen from the preceding cell, the percent of reads mapped by salmon, no matter which `tx2gene` file we use, is on average 3x higher than when we map to the genome and count reads that fall in feature regions.

Surprisingly, there's only an average of 11% gain by mapping to all transcripts (500k _more_ transcripts than is in the gene-only tx2gene file) than to just the gene-only transcripts.

Given these results, I'm leaning toward using the salmon counts with only the transcripts that mapped to genes represented. I don't think its worth the extra noise to include 500k more transcripts to gain 11% more read mapping. Note that if my de novo clustering (mmseqs2) had been more successful, I would have clustered these transcripts and then included those cluster representatives in my tx2gene file. However I got very little reduction with clustering (never dropped below 1M transcripts, even when dropping my percent identity to 70%), so I'm just going to ignore those transcripts for now.

## print session info

In [20]:
sessionInfo()

R version 4.3.0 (2023-04-21)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur ... 10.16

Matrix products: default
BLAS/LAPACK: /Users/taylorreiter/miniconda3/envs/tidyjupyter/lib/libopenblasp-r0.3.23.dylib;  LAPACK version 3.11.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/New_York
tzcode source: system (macOS)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] jsonlite_1.8.7  tximport_1.28.0 lubridate_1.9.2 forcats_1.0.0  
 [5] stringr_1.5.0   dplyr_1.1.2     purrr_1.0.1     readr_2.1.4    
 [9] tidyr_1.3.0     tibble_3.2.1    ggplot2_3.4.2   tidyverse_2.0.0

loaded via a namespace (and not attached):
 [1] bit_4.0.5        gtable_0.3.3     compiler_4.3.0   crayon_1.5.2    
 [5] tidyselect_1.2.0 IRdisplay_1.1    parallel_4.3.0   scales_1.2.1    
 [9] uuid_1.1-0       fastmap_1.1.1    IRkernel_1.3.2   R6_2.5.1        
[13] generics_0.